# Structured Outputs

This shows an example of generating structured outputs via an LLM for police narratives.

In [1]:
from src import bedrock # my own custom functions for working with AWS bedrock
import pandas as pd

# A simplified example, giving a description
system = '''I will give you a description of a burglary please return json that determines the method of entry, and the location of the entry
for example:

The offender pushed in the AC unit on the west side of the house.
return {"moe":"window", "loc": "side"}

The offender entered via the unlocked front door.
return {"moe":"unforced", "loc": "front"}

The burglary narrative is'''

# My class for using AWS bedrock
sonnet = bedrock.ClaudeModel(system=system)

# Example new burglary narrative
narr = "The offender broke in a window on the back door to enter the residence."

data = sonnet.struct_output(narr)
print(data)

{'moe': 'window', 'loc': 'back'}


In [2]:
# Slightly more complicated example, categorizing more crimes and more elements
narr_data = pd.read_csv('Narr.zip')
narr = "The offender broke in a window on the back door to enter the residence."
examples = pd.read_csv('SampleSet.csv')

el = []
for i,s,r,o in examples.itertuples():
    t = f"<offense_narrative>{o}</offense_narrative>"
    t += f"<output>{r}</output>"
    el.append(t)

shot = "\n".join(el)

prompt = f"""
{shot}

You will be analyzing an offense narrative and classifying various elements within it. Your task is to identify and categorize specific pieces of information from the narrative based on a provided list of elements.

crime, weapon, modus-operandi, vehicle-type, items

The narratives may contain other events that are not crimes, like traffic collisions or evidence collection. All narratives should have a crime type listed, but may not have the other types. Burglary and motor vehicle theft will sometimes have modus operandi. Shootings will sometimes have a type of gun listed, e.g. handgun or rifle. Larceny, theft from mv, and burglary will sometimes have specific items listed as stolen. Place those in a list.

Return the output in json format, only return the json, do not return any further description

<offense_narrative>
"""

print(prompt)

# Using the cheaper Haiku model
haiku = bedrock.ClaudeModel(system=prompt,model=bedrock.models['Claude 3.5 Haiku'])

# A test case, shows entire output not just parsed json
haiku.invoke(narr,assistant="</offense_narrative><output>")


<offense_narrative>SUSP TOOK COMP'S PROPERTY WITHOUT CONSENT. END OF ELEMENTS.</offense_narrative><output>{'crime': 'larceny'}</output>
<offense_narrative>INV OF BURGLARY. END OF ELEMENTS</offense_narrative><output>{'crime': 'burglary'}</output>
<offense_narrative>SEE RELATED RPT #101608B. ON LISTED DATE AND TIME, SUSP SHOT AT COMP'S  DIRECTION INTENDING TO ASSAULT HIM. END OF                              ELEMENTS.</offense_narrative><output>{'crime': 'shooting', 'weapon': 'gun'}</output>
<offense_narrative>BETWEEN THE HOURS OF 5:00PM AND 7:00PM AT THE OFFENSE LOCATION 3501     SAMUELL BLVD, TENSION PARK GOLF COURSE. THE COMP AND THE CC LEFT THEIR  BELONGS, 2 LAPTOP AND IPAD 4, IN TWO SEPARATE LAPTOP BAGS IN THE REAR   SEAT OF THE COMP'S VEHICLE A 2007 CHEVY SILVERADO. UNK SUSP PRIED OPEN  THE COMP'S REAR DRIVERSIDE WINDOW, MADE ENTRY, AND TOOK THE COMP'S AND  CC'S LISTED PROPERTY WITHOUT CONSENT. SUSP FLED IN UNK DIRECTION BY UNK MEANS.NF END OF ELEMENT</offense_narrative><output>{'c

"{'crime': 'burglary', 'modus-operandi': 'broke window on back door'}"

In [3]:
# Selecting the first 10 cases, and showing how this will work
res_data = []

for i,num,off_narr,off_date in narr_data.head(10).itertuples():
    print("")
    print(f'Row {i}')
    print(off_narr)
    data = haiku.invoke(off_narr,assistant="</offense_narrative><output>")
    print("")
    print(data)
    res_data.append(data)


Row 0
ON LISTED DATE AND TIME, THE COMPLAINANT WAS DISCOVERED DECEASED INSIDE HIS RESIDENCE. IT APPEARED THE COMPLAINANT WAS SHOT MULTIPLE TIMES.     THERE WAS EVIDENCE TO SUGGEST ROBBERY WAS THE MOTIVE BECAUSE THE        RESIDENCE WAS UNLOCKED AND THERE WAS AN                                 UNDETERMINED AMOUNT OF US CURRENCY FOUND JUST                           OUTSIDE THE COMPLAINANT'S FRONT DOOR.

{
    "crime": "murder",
    "weapon": "gun",
    "modus-operandi": "robbery"
}

Row 1
THE COMP STATED THAT ON THE LISTED DATE AND TIME, UNK SUSP(S) TOOK THE  LISTED VEH WITHOUT THE KEYS AND WITHOUT PERMISSION. THE VEH WAS PARKED  ON THE STREET, IN FRONT OF COMP'S RESIDENCE. TOW/REPO WAS NEGATIVE. THE COMP IS THE OWNER, HAS TITLE, BUT THE VEH IS STILL UNDER THE ORIGINAL   OWNER'S NAME. COMP FILLED OUT AN AUTO THEFT STOLEN VEH AFFIDAVIT.       NFI. END OF ELEMENTS

{
    "crime": "mv theft"
}

Row 2
THE COMP. STATED THAT HE LAST SAW HIS MOTORCYCLE IN FRONT OF HIS        APARTMENT AROUND 8

# Estimating Cost

So using these public services are not free. You could use local models to do this as well (for most current, need a 16 gig GPU or Mac with a M1 processor). The [current costs](https://aws.amazon.com/bedrock/pricing/) for Haiku using AWS like I have is `0.0008` per 1000 tokens input, and `0.004` per 1000 tokens output.

In my methods, I accumulate those tokens, so we can see what the average cost is for these 11 samples I have generated

In [4]:
cost_input = 0.0008/1000
cost_output = 0.004/1000

token_usage = pd.DataFrame(haiku.invoke_history,columns=['text','input_tokens','output_tokens','response_text','begin','end'])
cost_input = token_usage['input_tokens'].mean()*cost_input
cost_output = token_usage['output_tokens'].mean()*cost_output
cost_input + cost_output # average cost per row

np.float64(0.0012037090909090909)

There are additional ways to bring this cost down:

 - using batch operations cuts cost in half
 - if the system prompt is greater than 1000 tokens, you can cache the system prompt (it is why I write the system prompt this way, with k-shot examples first, instructions second, and then the actual value last)

With caching on AWS, you will not always hit the cache, but it can additionally reduce costs by a significant amount (almost to 10% of original).